# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("city_weather.csv")
weather_df.head()


,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind
0,6167817,Torbay,CA,47.6666,-52.7314,69.04,67,81,17.31
1,2729907,Longyearbyen,SJ,78.2186,15.6401,42.64,87,75,12.66
2,964420,Port Elizabeth,ZA,-33.9180,25.5701,59.09,88,100,8.12
3,5905393,Bonavista,CA,48.6499,-53.1147,71.60,76,100,7.70
4,1259385,Port Blair,IN,11.6667,92.7500,81.27,83,100,19.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Use Lat and Lng as locations and Humidity as weight
lat_lon = weather_df[['Lat', 'Lng']]

humidity = weather_df["Humidity"]
lat_lon.head()

,Lat,Lng
0,47.6666,-52.7314
1,78.2186,15.6401
2,-33.9180,25.5701
3,48.6499,-53.1147
4,11.6667,92.7500


In [10]:
#Add Heatmap layer to map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1 px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

#Use gmaps.figure to reconfigure size
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

#Create and add heat layer
heat_layer = gmaps.heatmap_layer(lat_lon, weights=humidity, dissipating=False, max_intensity=100, point_radius=1.5)

fig.add_layer(heat_layer)
plt.savefig("Heat map.png")
fig


Figure(layout=FigureLayout(border='1 px solid black', height='600px', margin='0 auto 0 auto', padding='1px', w…

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create criteria for ideal weather condition

#Max temp lower than 80 degrees, but higher than 70 degrees
ideal_temp = (weather_df.Temp < 80) & (weather_df.Temp > 70)
#ideal wind speed less than 10 mph
ideal_wind = weather_df.Wind < 10
#Zero cloudiness
ideal_cloud = weather_df.Clouds == 0

ideal_weather = ideal_temp & ideal_wind & ideal_cloud

#Use boolean indexing to drop rows with null values
ideal_weather_df = weather_df[ideal_weather]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head(10)

,index,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind
0,28,3372707,Ribeira Grande,PT,38.5167,-28.7000,73.60,87,0,6.31
1,90,1513087,Nurota,UZ,40.5614,65.6886,77.09,22,0,6.31
2,120,5919850,Chapais,CA,49.7834,-74.8492,73.18,32,0,6.55
3,169,921786,Mitsamiouli,KM,-11.3847,43.2844,74.88,73,0,2.06
4,241,1847947,Shingū,JP,33.7333,135.9833,79.92,76,0,3.40
5,252,2236967,Soyo,AO,-6.1349,12.3689,72.36,90,0,9.24
6,283,305742,Kulu,TR,39.0951,33.0799,76.17,30,0,8.12
7,318,314188,Göksun,TR,38.0210,36.4973,76.15,43,0,5.79
8,483,5148635,Bucyrus,US,40.8084,-82.9755,78.69,50,0,6.29
9,521,3351663,Benguela,AO,-12.5763,13.4055,70.11,87,0,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store variable named hotel_df
hotel_df = ideal_weather_df

#add column for Hotel Name
hotel_df['Hotel Name'] = ""
hotel_df.head()

,index,City ID,City,Country,Lat,Lng,Temp,Humidity,Clouds,Wind,Hotel Name
0,28,3372707,Ribeira Grande,PT,38.5167,-28.7000,73.60,87,0,6.31,
1,90,1513087,Nurota,UZ,40.5614,65.6886,77.09,22,0,6.31,
2,120,5919850,Chapais,CA,49.7834,-74.8492,73.18,32,0,6.55,
3,169,921786,Mitsamiouli,KM,-11.3847,43.2844,74.88,73,0,2.06,
4,241,1847947,Shingū,JP,33.7333,135.9833,79.92,76,0,3.40,


In [7]:
#Set parameters to search hotels w/in 5000 m
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

#Use latitude and longitude from hotel_df to ID airports
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
#Change location for each iteration while keeping original original params
    params["location"] = f"{lat},{lng}"

#Hit the Google places API for each city's coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

city_coord_url = requests.get(base_url, params=params)

#jsonify url
city_coord_json = city_coord_url.json()

print(json.dumps(city_coord_json, indent=4, sort_keys=True))
#needed to import json

#Store the first Hotel result into the DataFrame

#Plot markers on top of the heatmap

{
    "error_message": "This API project is not authorized to use this API.",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1 px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15,25))

#Create symbol layer for hotel
hotel_layer = gmaps.marker_layer(locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()])

#Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
plt.savefig("Hotel Layer Heat map.png")
# Display figure
fig



Figure(layout=FigureLayout(border='1 px solid black', height='600px', margin='0 auto 0 auto', padding='1px', w…

<Figure size 432x288 with 0 Axes>